# Getting started with Spectral Radius Optimization

This notebook contains examples of how to solve Spectral Radius Optimization problem.

For more details, please check the documentation website https://pygranso.readthedocs.io/en/latest/

1. Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import torch
import os,sys
## Adding PyGRANSO directories. Should be modified by user
sys.path.append('/home/buyun/Documents/GitHub/PyGRANSO')
from pygranso import pygranso
from pygransoStruct import Options, Data, GeneralStruct 
import scipy.io
from torch import linalg as LA

2. Specify torch device, and read the data from provided file

In [2]:
device = torch.device('cpu')

# currentdir = os.path.dirname(os.path.realpath(__file__))
file = "/home/buyun/Documents/GitHub/PyGRANSO/examples/spec_radius_opt_data.mat"
mat = scipy.io.loadmat(file)
mat_struct = mat['sys']
mat_struct = mat_struct[0,0]
A = torch.from_numpy(mat_struct['A']).to(device=device, dtype=torch.double)
B = torch.from_numpy(mat_struct['B']).to(device=device, dtype=torch.double)
C = torch.from_numpy(mat_struct['C']).to(device=device, dtype=torch.double)
p = B.shape[1]
m = C.shape[0]
stability_margin = 1

3. Spceify optimization variables and corresponding objective and constrained function.

Note: please strictly follow the format of evalObjFunction and combinedFunction, which will be used in the PyGRANSO main algortihm. *X_struct* and *data_in* are always required.

In [3]:
# variables and corresponding dimensions.
var_in = {"X": [p,m] }


def evalObjFunction(X_struct,data_in = None):
    # user defined variable, matirx form. torch tensor
    X = X_struct.X
    X.requires_grad_(True)

    # objective function
    M           = A + B@X@C
    [D,_]       = LA.eig(M)
    f = torch.max(D.imag)
    return f

def combinedFunction(X_struct,data_in = None):
    # user defined variable, matirx form. torch tensor
    X = X_struct.X
    X.requires_grad_(True)

    # objective function
    M           = A + B@X@C
    [D,_]       = LA.eig(M)
    f = torch.max(D.imag)

    # inequality constraint, matrix form
    ci = GeneralStruct()
    ci.c1 = torch.max(D.real) + stability_margin

    # equality constraint 
    ce = None
    
    return [f,ci,ce]

obj_eval_fn = lambda X_struct,data_in = None : evalObjFunction(X_struct,data_in = None)
comb_fn = lambda X_struct,data_in = None : combinedFunction(X_struct,data_in = None)

4. Specify user-defined options for PyGRANSO algorithm

In [4]:
opts = Options()
opts.QPsolver = 'osqp' 
opts.maxit = 200
opts.x0 = torch.zeros(p*m,1).to(device=device, dtype=torch.double)
opts.print_level = 1
opts.print_frequency = 10
opts.limited_mem_size = 40

4. Run main algorithm

In [5]:
start = time.time()
soln = pygranso(combinedFunction = comb_fn, objEvalFunction = obj_eval_fn,var_dim_map = var_in, torch_device = device, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))

/home/buyun/anaconda3/envs/cuosqp_pygranso/lib/python3.9/site-packages/torch/autograd/__init__.py:147: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/cuda/CUDAFunctions.cpp:115.)
  Variable._execution_engine.run_backward(
/home/buyun/anaconda3/envs/cuosqp_pygranso/lib/python3.9/site-packages/osqp/utils.py:122: UserWarning: Converting sparse A to a CSC (compressed sparse column) matrix. (It may take a while...)
  warn("Converting sparse A to a CSC " +




╔═════ QP SOLVER NOTICE ══════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible   ║
║  interface, as defined by osqp and Gurobi...                                        ║
╚═════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
Problem specifications:                                                                                          ║ 
 # of variables                     :   200                                                                      ║ 
 # of inequality constraints        :     1                                                                      ║ 
 # of equality constraints          :     0                                                                      ║ 
═══════════════════════════════════════════════════════════════════

/home/buyun/Documents/GitHub/PyGRANSO/private/bfgsHessianInverseLimitedMem.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.rho = torch.hstack((self.rho[:,1:], torch.tensor(rho_new)))


  50 ║ 1.000000 │  12.4390566671 ║  11.9740978429 ║ 0.464959 │   -  ║ S  │     5 │ 0.062500 ║     1 │ 0.056390   ║ 
  60 ║ 0.900000 │  11.0818212983 ║  11.9436807004 ║ 0.332509 │   -  ║ S  │     5 │ 0.062500 ║     1 │ 0.026745   ║ 
  70 ║ 0.729000 │  8.88435648229 ║  11.8258869667 ║ 0.263285 │   -  ║ S  │     4 │ 0.125000 ║     1 │ 0.192494   ║ 
  80 ║ 0.729000 │  8.76098876216 ║  11.7289106462 ║ 0.210613 │   -  ║ S  │     7 │ 0.015625 ║     1 │ 0.060382   ║ 
  90 ║ 0.729000 │  8.72408094526 ║  11.7028130102 ║ 0.192730 │   -  ║ S  │     8 │ 0.007812 ║     1 │ 0.108132   ║ 
 100 ║ 0.729000 │  8.67882291586 ║  11.6772329705 ║ 0.166120 │   -  ║ S  │     9 │ 0.003906 ║     1 │ 0.548152   ║ 
 110 ║ 0.729000 │  8.64383040019 ║  11.6616361053 ║ 0.142498 │   -  ║ S  │    11 │ 9.77e-04 ║     1 │ 1.491162   ║ 
 120 ║ 0.656100 │  7.75190289655 ║  11.6443673345 ║ 0.112033 │   -  ║ S  │    19 │ 3.81e-06 ║     1 │ 1.175629   ║ 
 130 ║ 0.531441 │  6.27872545056 ║  11.6323402041 ║ 0.096823 │   -  ║ S 